In [3]:
import sqlite3
import os
import json
from datetime import datetime, timedelta

# --- Configuration ---
ROOMS_FILE = "hospital_rooms.json"
PATIENTS_FILE = "hospital_patients.json"
SURGERY_DB_FILE = "surgery_comments.db"

# --- Sample Data ---
# Create timestamps relative to the current time for realism
now = datetime.now()

sample_patients = [
  {
    "pid": 101,
    "time_of_admit": (now - timedelta(days=1, hours=8)).isoformat(), # Admitted yesterday
    "room_name": "A",
    "bed_number": 1,
    "first_name": "John",
    "last_name": "Doe",
    "dob": "1985-03-15",
    "description": "Appendicitis"
  },
  {
    "pid": 102,
    "time_of_admit": (now - timedelta(days=1, hours=6)).isoformat(), # Admitted yesterday
    "room_name": "A",
    "bed_number": 2,
    "first_name": "Jane",
    "last_name": "Smith",
    "dob": "1992-11-20",
    "description": "Compound Fracture (Tibia)"
  },
  {
    "pid": 103,
    "time_of_admit": (now - timedelta(days=1, hours=2)).isoformat(), # Admitted yesterday
    "room_name": "F",
    "bed_number": 1,
    "first_name": "Robert",
    "last_name": "Brown",
    "dob": "1978-07-01",
    "description": "Gallbladder Stones"
  }
]

# Create rooms, ensuring sample patients occupy the correct beds
sample_rooms = []
room_id_counter = 1
for room_letter in ["A", "B", "C", "D", "E", "F"]:
    for bed_num in [1, 2]:
        occupied_status = "no"
        assigned_pid = None
        for p in sample_patients:
            if p["room_name"] == room_letter and p["bed_number"] == bed_num:
                occupied_status = "yes"
                assigned_pid = p["pid"]
                break
        sample_rooms.append({
            "id": room_id_counter,
            "room_name": room_letter,
            "bed_number": bed_num,
            "occupied": occupied_status,
            "pid": assigned_pid
        })
        room_id_counter += 1

# --- More Detailed Transcripts ---

transcript_101_surgery = """
[SPEAKER 0 Dr. Aris] Okay team, let's begin. Time noted, 09:12 start. Scalpel.
[SPEAKER 1 Nurse] Scalpel.
[SPEAKER 0 Dr. Aris] Making the initial incision... Laparoscope in. Let's visualize.
[SPEAKER 2 Anesthesiologist] Vitals stable: BP 118 over 76, pulse 78, O2 99%. Patient is well-anesthetized.
[SPEAKER 0 Dr. Aris] Good view. Appendix is clearly inflamed, significant edema at the base, but no sign of perforation yet. Grasper.
[SPEAKER 1 Nurse] Grasper.
[SPEAKER 0 Dr. Aris] We'll proceed with the standard laparoscopic appendectomy. Isolating the mesoappendix now. Cautery.
[SPEAKER 1 Nurse] Cautery ready.
[SPEAKER 0 Dr. Aris] Okay, mesoappendix divided. Applying endoloop ligature to the base... Tight... Secure. Cutter.
[SPEAKER 1 Nurse] Cutter.
[SPEAKER 0 Dr. Aris] Appendix transected. Specimen retrieval bag.
[SPEAKER 1 Nurse] Bag ready.
[SPEAKER 0 Dr. Aris] Appendix secured in the bag. Removing now... Irrigation. Let's get a good wash. Suction.
[SPEAKER 2 Anesthesiologist] Slight transient drop in BP during manipulation, now 110 over 70, pulse 82. Stable otherwise.
[SPEAKER 0 Dr. Aris] Noted. Peritoneal cavity looks clear, no residual pus or contamination. Minimal bleeding throughout. Looks good. We can start closing. Time check?
[SPEAKER 1 Nurse] 09:41.
[SPEAKER 0 Dr. Aris] Excellent. Port sites closed. Final count correct. Procedure complete.
"""

transcript_101_followup = """
[SPEAKER 0 Dr. Aris] Morning round for patient 101, John Doe. Post-op day one appendectomy. How are you feeling, John? Pain manageable?
[SPEAKER 1 Nurse] Patient reports pain level 3 out of 10. Received scheduled analgesia one hour ago. Tolerating sips of clear fluids. Bowel sounds present but hypoactive. Incision sites clean, dry, intact. Vitals stable overnight, afebrile.
[SPEAKER 0 Dr. Aris] Good. Ambulation encouraged today. Advance diet as tolerated to full liquids. Continue monitoring vitals and incision sites. Any nausea?
[SPEAKER 1 Nurse] Minimal nausea reported, resolved after fluids.
[SPEAKER 0 Dr. Aris] Okay, looks like good progress. We'll reassess this afternoon.
"""

transcript_102_surgery = """
[SPEAKER 0 Dr. Bevan] Alright, patient 102, compound tibia fracture. X-rays show significant displacement. We'll perform an open reduction and internal fixation with an intramedullary nail. Anesthesia, status?
[SPEAKER 1 Anesthesiologist] Patient is stable, vitals strong. Ready for incision.
[SPEAKER 0 Dr. Bevan] Scalpel.
[SPEAKER 2 Nurse] Scalpel.
[SPEAKER 0 Dr. Bevan] Incision made over the fracture site... Good exposure. Significant soft tissue damage as expected. Irrigation. Lots of it.
[SPEAKER 2 Nurse] Irrigating now.
[SPEAKER 0 Dr. Bevan] Okay, debriding non-viable tissue... Careful retraction. Now reducing the fracture fragments... Alignment looks good. Guide wire in. Reaming.
[SPEAKER 1 Anesthesiologist] BP holding steady at 125 over 80, pulse 70.
[SPEAKER 0 Dr. Bevan] Intramedullary nail inserted. Proximal locking screws... Distal locking screws... Confirming position with C-arm... Perfect placement. Excellent stability. Let's close. Final irrigation.
[SPEAKER 2 Nurse] Closing sutures ready.
[SPEAKER 0 Dr. Bevan] Wound closed in layers. Dressing applied. Splint will be placed. Procedure complete. Time?
[SPEAKER 2 Nurse] 11:15.
[SPEAKER 0 Dr. Bevan] Good work team. Post-op X-rays and neurovascular checks are crucial.
"""

transcript_102_discharge_note = """
[SPEAKER 0 Dr. Bevan] Patient 102, Jane Smith, post-op day 3, ORIF tibia fracture. Patient ambulating well with crutches, weight-bearing as tolerated per physio. Pain well controlled on oral analgesics. Incision clean, dry, intact, no signs of infection. Neurovascular status intact distally. Patient understands wound care and follow-up instructions. Fit for discharge home with outpatient physiotherapy arranged. Prescriptions provided.
"""

transcript_103_surgery = """
[SPEAKER 0 Dr. Cygnus] Patient 103, cholecystectomy for symptomatic gallstones. Starting laparoscopy. Trocar placement... CO2 insufflation initiated. Camera in.
[SPEAKER 1 Nurse] Pressure stable at 14 mmHg.
[SPEAKER 0 Dr. Cygnus] Visualizing the gallbladder... Ooh, quite a bit of inflammation around Calot's triangle, as expected from the ultrasound. Lots of adhesions. Dissector.
[SPEAKER 1 Nurse] Dissector.
[SPEAKER 0 Dr. Cygnus] Carefully dissecting the adhesions... Need a clear view of the cystic duct and artery. Grasper.
[SPEAKER 1 Nurse] Grasper.
[SPEAKER 2 Anesthesiologist] Patient's BP trending up, 155 over 90. Pulse 95.
[SPEAKER 0 Dr. Cygnus] Noted. Any concerns?
[SPEAKER 2 Anesthesiologist] Administering additional medication to manage hypertension. Watching closely.
[SPEAKER 0 Dr. Cygnus] Okay, cystic duct identified... Clipped proximally and distally. Cystic artery identified... Clipped. Division... Good. Gallbladder detached from the liver bed using cautery. Minimal bleeding.
[SPEAKER 2 Anesthesiologist] BP settling nicely, 130 over 80 now.
[SPEAKER 0 Dr. Cygnus] Excellent. Retrieval bag.
[SPEAKER 1 Nurse] Bag deployed.
[SPEAKER 0 Dr. Cygnus] Gallbladder secured. Removing specimen... Irrigation and suction. Checking for any bile leak or bleeding... Looks clean. We can remove ports and close. Time?
[SPEAKER 1 Nurse] 14:05.
[SPEAKER 0 Dr. Cygnus] Procedure complete. Send specimen to pathology.
"""

sample_surgery_comments = [
    # Patient 101
    ('101', 'Dr. Aris', transcript_101_surgery.strip(), (now - timedelta(days=1)).isoformat()), # Surgery yesterday
    ('101', 'Dr. Aris', transcript_101_followup.strip(), (now - timedelta(hours=2)).isoformat()), # Follow-up today
    # Patient 102
    ('102', 'Dr. Bevan', transcript_102_surgery.strip(), (now - timedelta(days=1, hours=1)).isoformat()), # Surgery yesterday
    ('102', 'Dr. Bevan', transcript_102_discharge_note.strip(), (now - timedelta(minutes=30)).isoformat()), # Discharge note today
    # Patient 103
    ('103', 'Dr. Cygnus', transcript_103_surgery.strip(), (now - timedelta(hours=6)).isoformat()) # Surgery today
]

# --- Database Population Functions ---

def setup_json_files():
    """Writes the sample data to the JSON files, overwriting existing ones."""
    try:
        with open(PATIENTS_FILE, 'w') as f:
            json.dump(sample_patients, f, indent=2)
        print(f"✅ Wrote sample data to {PATIENTS_FILE}")

        with open(ROOMS_FILE, 'w') as f:
            json.dump(sample_rooms, f, indent=2)
        print(f"✅ Wrote sample data to {ROOMS_FILE}")
    except Exception as e:
        print(f"❌ Error writing JSON files: {e}")

def setup_surgery_db():
    """Creates/overwrites the DB and inserts sample surgery comments."""
    if os.path.exists(SURGERY_DB_FILE):
        os.remove(SURGERY_DB_FILE)
        print(f"Removed old database file: {SURGERY_DB_FILE}")

    conn = None
    try:
        conn = sqlite3.connect(SURGERY_DB_FILE)
        c = conn.cursor()
        # Create table WITHOUT primary key on patientid, add rowid automatically
        c.execute("""
            CREATE TABLE IF NOT EXISTS surgery_comments (
                comment_id INTEGER PRIMARY KEY AUTOINCREMENT, -- Added unique ID
                patientid TEXT NOT NULL, -- Keep patient ID
                doctor TEXT,
                comment TEXT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        """)
        print("Table 'surgery_comments' created or already exists.")

        # Insert data using simple INSERT (allows multiple rows per patientid)
        # Adjusting the query to match the new schema (excluding comment_id)
        insert_query = """
            INSERT INTO surgery_comments (patientid, doctor, comment, timestamp)
            VALUES (?, ?, ?, ?)
        """
        # Prepare data tuples matching the insert query order
        data_to_insert = [
            (pid, doc, comm, ts) for pid, doc, comm, ts in sample_surgery_comments
        ]
        c.executemany(insert_query, data_to_insert)
        conn.commit()
        print(f"✅ Success! Database '{SURGERY_DB_FILE}' created and populated with {len(data_to_insert)} records.")

    except sqlite3.Error as e:
        print(f"❌ An error occurred with the surgery DB: {e}")
        if conn:
            conn.rollback() # Rollback changes if error occurs
    finally:
        if conn:
            conn.close()

# --- Main Execution ---
if __name__ == "__main__":
    print("Starting data population...")
    setup_json_files()
    setup_surgery_db()
    print("\nData population complete.")

Starting data population...
✅ Wrote sample data to hospital_patients.json
✅ Wrote sample data to hospital_rooms.json
Table 'surgery_comments' created or already exists.
✅ Success! Database 'surgery_comments.db' created and populated with 5 records.

Data population complete.
